# Customize Chat model
* reference
    * https://langchain.readthedocs.io/en/latest/modules/agents/examples/chat_conversation_agent.html
    https://note.com/npaka/n/n6b7a07e492f1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain/blob/main/examples/customize_chat_model.ipynb)

In [4]:
%%capture
!pip install langchain
!pip install google-search-results
!pip install openai
!pip install wikipedia

In [5]:
from langchain.utilities import SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.agents import load_tools
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent

In [ ]:
import os

os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""

### tool一覧
* python_relp
    * pythonコマンドを実行できる．
* serpapi
    * サーチエンジン，現在のイベントに関する質問に答える必要がある場合に役に立つ
* requests
    * サイトから特定のコンテンツを取得する場合に使う
    * 入力はURLで出力はそのページの全てのテキスト
* terminal
    * ターミナルでのコマンド実行
* llm-math
    * 数学の計算に答えるよう
* pal-math
    * 複雑な数学の問題に答えるのに優れた言語モデル
* pal-colored-objects
    * オブジェクトの位置と色についての推論に優れた言語モデル
* open-meteo-api
    * 気象情報を取得したい場合に使う
* news-api
    * 現在のニュース記事のトップ，ヘッドラインに関する情報を取得する
* tmdb-api
    * The Movie Databaseから情報を取得するのに使う

In [6]:
llm = OpenAI(temperature=0)

tools = load_tools(
    ["python_repl",
    "serpapi",
    "requests",
    "terminal",
    "pal-math",
    "pal-colored-objects",
    "llm-math",
    "open-meteo-api",
    #"news-api",
    #"tmdb-api"
    ], 
    llm=llm)

### agent一覧
* どのアクションをどのような順序で行うかを決めるもの
* zero-shot-react-description
    * 複数のツールが使用できるエージェント
    * ツールの説明に基づいて使うものを決める，ツールには説明文が必ず必要になる
* react-docstore
    * SearchとLookupの二つのツールを使う．
* self-ask-with-search
    * intermediate Answerという一つのツールのみ使う
* conversational-react-description
    * 会話用に最適化されたエージェント，以前の会話を記憶するためメモリが必要になる
    * エージェントとユーザーが会話を行い必要な時にツールを呼び出す
    * [実装例](https://github.com/fuyu-quant/langchain/blob/main/examples/search_chat_model.ipynb)

### react-docstoreを使った実装
* https://langchain.readthedocs.io/en/latest/modules/agents/implementations/react.html

In [12]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

In [13]:
docstore=DocstoreExplorer(Wikipedia())


tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]


llm = OpenAI(temperature=0, model_name="text-davinci-002")


agent = initialize_agent(
    tools = tools,
    llm = llm, 
    agent="react-docstore", 
    verbose=True
    )

In [14]:
question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"

agent.run(question)



> Entering new AgentExecutor chain...

Thought 1: I need to search David Chanoff and find the U.S. Navy admiral he collaborated
with.
Action 1: Search[David Chanoff]
Observation 1: David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.
Thought 2: The U.S. Navy admiral David Chanoff collaborated with is William J. Crowe.
Action 2: Search[William J. Crowe]
Observation 2: William James Crowe Jr. (January 2, 1925 – October 18, 2007) was a United States Navy admiral and diplomat who served as the 11th chairman of the Joint Chiefs of 

 The President William J. Crowe served as the ambassador to the United Kingdom under is Bill Clinton.
Action 3: Finish[Bill Clinton]

> Finished chain.


'Bill Clinton'

### self-ask-with-searchを使った実装